In [4]:
#!pip install pyodbc

import pyodbc
import pandas as pd
import os

  Using cached pyodbc-4.0.39-cp310-cp310-win_amd64.whl (69 kB)


You should consider upgrading via the 'C:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF\.venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [2]:
"""
Connection-Info
Rechnername:    IPEPDVMSSQLDB2.ipe.kit.edu
IP-Adresse:    141.52.64.73
SQL-Server-Port:    1433
Datenbank:    EnMaSy

SQL-Server-Authentifizierung für "SmartMeter / $Smart$Meter$"
"""

'\nConnection-Info\nRechnername:    IPEPDVMSSQLDB2.ipe.kit.edu\nIP-Adresse:    141.52.64.73\nSQL-Server-Port:    1433\nDatenbank:    EnMaSy\n\nSQL-Server-Authentifizierung für "SmartMeter / $Smart$Meter$"\n'

In [9]:
"""# get current working directory and go back one folder to main working directory
cwd = os.path.normpath(os.getcwd())
# set path to load data file
load_data_path = '/d01_weather_data_all_raw/Data_weather_meteostat_2019-01-01 bis 2021_12-31.csv'
path = cwd + load_data_path
  
#Read CSV file to pandas dataframe; encoding= 'unicode_escape': Decode from Latin-1 source code. Default UTF-8.
df = pd.read_csv(path, encoding= 'unicode_escape', index_col='Date')
df"""

"# get current working directory and go back one folder to main working directory\ncwd = os.path.normpath(os.getcwd())\n# set path to load data file\nload_data_path = '/d01_weather_data_all_raw/Data_weather_meteostat_2019-01-01 bis 2021_12-31.csv'\npath = cwd + load_data_path\n  \n#Read CSV file to pandas dataframe; encoding= 'unicode_escape': Decode from Latin-1 source code. Default UTF-8.\ndf = pd.read_csv(path, encoding= 'unicode_escape', index_col='Date')\ndf"

In [6]:
driver = 'ODBC Driver 18 for SQL Server' #Must by downloaded and installed in advance
server = 'IPEPDVMSSQLDB2.ipe.kit.edu,1433'
database = 'EnMaSy'
username= 'SmartMeter'
password ='$Smart$Meter$'


cnxn = pyodbc.connect(
    f'DRIVER={driver};'
    f'SERVER={server};'
    f'DATABASE={database};'
    f'ENCRYPT=yes;'
    f'UID={username};'
    f'PWD={password};'
    f'TrustServerCertificate=yes;'
)
cursor = cnxn.cursor()


In [8]:

cursor.execute(
    '''
    CREATE TABLE sensorNodeThData (
        id INT NOT NULL IDENTITY PRIMARY KEY,
        date DATETIME,
        temp FLOAT(1),
        pres FLOAT(1),
        sound FLOAT(1),
        light FLOAT(1),
    )
    '''
)

cnxn.commit()


In [5]:
"""
cursor.execute(
    '''
    CREATE TABLE weatherData (
        id INT NOT NULL IDENTITY PRIMARY KEY,
        date DATETIME,
        temp FLOAT(1),
        dwpt FLOAT(1),
        rhum FLOAT(1),
        prcp FLOAT(1),
        wdir FLOAT(1),
        wspd FLOAT(1),
        wpgt FLOAT(1),
        pres FLOAT(1),
        coco FLOAT(1)
    )
    '''
)

cnxn.commit()
"""

In [11]:
sql = '''ALTER TABLE weatherData
ALTER COLUMN date VARCHAR(25);'''

cursor.execute(sql)
cnxn.commit()

In [12]:
# Insert Dataframe into SQL Server:
for index, row in df.iterrows():
     cursor.execute(
         "INSERT INTO weatherData ("
         "date,temp,dwpt, rhum, prcp, wdir, wspd, wpgt, pres, coco) values(?,?,?,?,?,?,?,?,?,?)",
         index, row.temp, row.dwpt, row.rhum, row.prcp, row.wdir,
         row.wspd, row.wpgt, row.pres,row.coco
     )
cnxn.commit()
cursor.close()